# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 51 new papers today
          18 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/17 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2303.08150


extracting tarball to tmp_2303.08150...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.08170


extracting tarball to tmp_2303.08170...

...

extracting tarball to tmp_2303.08643...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.08749


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2303.08643/new.ms.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2303.08749...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.08174-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.08174) | **A super-Earth and a mini-Neptune near the 2:1 MMR straddling the radius  valley around the nearby mid-M dwarf TOI-2096**  |
|| F. J. Pozuelos, et al. -- incl., <mark>R. D. Wells</mark>, <mark>G. Torres</mark> |
|*Appeared on*| *2023-03-16*|
|*Comments*| *25 pages, 21 figures. Aceptted for publication in Astronomy & Astrophysics*|
|**Abstract**| Several planetary formation models have been proposed to explain the observed abundance and variety of compositions of super-Earths and mini-Neptunes. In this context, multitransiting systems orbiting low-mass stars whose planets are close to the radius valley are benchmark systems, which help to elucidate which formation model dominates. We report the discovery, validation, and initial characterization of one such system, TOI-2096, composed of a super-Earth and a mini-Neptune hosted by a mid-type M dwarf located 48 pc away. We first characterized the host star by combining different methods. Then, we derived the planetary properties by modeling the photometric data from TESS and ground-based facilities. We used archival data, high-resolution imaging, and statistical validation to support our planetary interpretation. We found that TOI-2096 corresponds to a dwarf star of spectral type M4. It harbors a super-Earth (R$\sim1.2 R_{\oplus}$) and a mini-Neptune (R$\sim1.90 R_{\oplus}$) in likely slightly eccentric orbits with orbital periods of 3.12 d and 6.39 d, respectively. These orbital periods are close to the first-order 2:1 mean-motion resonance (MMR), which may lead to measurable transit timing variations (TTVs). We computed the expected TTVs amplitude for each planet and found that they might be measurable with high-precision photometry delivering mid-transit times with accuracies of $\lesssim$2 min. Moreover, measuring the planetary masses via radial velocities (RVs) is also possible. Lastly, we found that these planets are among the best in their class to conduct atmospheric studies using the James Webb Space Telescope (JWST). The properties of this system make it a suitable candidate for further studies, particularly for mass determination using RVs and/or TTVs, decreasing the scarcity of systems that can be used to test planetary formation models around low-mass stars. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.08150-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.08150) | **TRINITY II: The Luminosity-dependent Bias of the Supermassive Black Hole  Mass--Galaxy Mass Relation for Bright Quasars at $z=6$**  |
|| <mark>Haowen Zhang</mark>, et al. -- incl., <mark>Joseph Silk</mark> |
|*Appeared on*| *2023-03-16*|
|*Comments*| *5 pages, 3 figures. Submitted to MNRAS Letters. Comments welcome!*|
|**Abstract**| Using recent empirical constraints on the dark matter halo--galaxy--supermassive black hole (SMBH) connection from $z=0-7$, we infer how undermassive, typical, and overmassive SMBHs contribute to the quasar luminosity function (QLF) at $z=6$. We find that beyond $L_\mathrm{bol} = 5 \times 10^{46}$ erg/s, the $z=6$ QLF is dominated by SMBHs that are at least 0.3 dex above the $z=6$ median $M_\bullet-M_*$ relation. The QLF is dominated by typical SMBHs (i.e., within $\pm 0.3$ dex around the $M_\bullet-M_*$ relation) at $L_\mathrm{bol} \lesssim 10^{45}$ erg/s. At $z\sim 6$, the intrinsic $M_\bullet-M_*$ relation for all SMBHs is slightly steeper than the $z=0$ scaling, with a similar normalization at $M_* \sim 10^{11} M_\odot$. We also predict the $M_\bullet-M_*$ relation for $z=6$ bright quasars selected by different bolometric luminosity thresholds, finding very good agreement with observations. For quasars with $L_\mathrm{bol} > 3 \times 10^{46}$ ($10^{48}$) erg/s, the scaling relation is shifted upwards by $\sim0.35$ (1.0) dex for $10^{11} M_\odot$ galaxies. To accurately measure the intrinsic $M_\bullet-M_*$ relation, it is essential to include fainter quasars with $L_\mathrm{bol} \lesssim 10^{45}$ erg/s. At high redshifts, low-luminosity quasars are thus the best targets for understanding typical formation paths for SMBHs in galaxies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.08198-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.08198) | **Dynamical Masses and Ages of Sirius-like Systems**  |
|| Hengyue Zhang, et al. -- incl., <mark>Yiting Li</mark> |
|*Appeared on*| *2023-03-16*|
|*Comments*| *34 pages, 21 figures. Submitted to MNRAS*|
|**Abstract**| We measure precise orbits and dynamical masses and derive age constraints for six confirmed and one candidate Sirius-like systems, including the Hyades member HD 27483. Our orbital analysis incorporates radial velocities, relative astrometry, and Hipparcos-Gaia astrometric accelerations. We constrain the main-sequence lifetime of a white dwarf's progenitor from the remnant's dynamical mass and semi-empirical initial-final mass relations and infer the cooling age from mass and effective temperature. We present new relative astrometry of HD 27483 B from Keck/NIRC2 observations and archival HST data, and obtain the first dynamical mass of ${0.798}_{-0.041}^{+0.10}$ $M_{\odot}$, and an age of ${450}_{-180}^{+570}$ Myr, consistent with previous age estimates of Hyades. We also measure precise dynamical masses for HD 114174 B ($0.591 \pm 0.011$ $M_{\odot}$) and HD 169889 B (${0.526}_{-0.037}^{+0.039}$ $M_{\odot}$), but their age precisions are limited by their uncertain temperatures. For HD 27786 B, the unusually small mass of $0.443 \pm 0.012$ $M_{\odot}$ suggests a history of rapid mass loss, possibly due to binary interaction in its progenitor's RGB phase. The orbits of HD 118475 and HD 136138 from our RV fitting are overall in good agreement with Gaia DR3 astrometric two-body solutions, despite moderate differences in the eccentricity and period of HD 136138. The mass of ${0.580}_{-0.039}^{+0.052}$ $M_{\odot}$ for HD 118475 B and a speckle imaging non-detection confirms that the companion is a white dwarf. Our analysis shows examples of a rich number of precise WD dynamical mass measurements enabled by Gaia DR3 and later releases, which will improve empirical calibrations of the white dwarf initial-final mass relation. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.08472-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.08472) | **Equalizing the Pixel Response of the Imaging Photoelectric Polarimeter  On-Board the IXPE Mission**  |
|| John Rankin, et al. -- incl., <mark>Fei Xie</mark> |
|*Appeared on*| *2023-03-16*|
|*Comments*| *Accepted for publication in The Astronomical Journal. 10 pages, 19 figures*|
|**Abstract**| The Gas Pixel Detector is a gas detector, sensitive to the polarization of X-rays, currently flying on-board IXPE - the first observatory dedicated to X-ray polarimetry. It detects X-rays and their polarization by imaging the ionization tracks generated by photoelectrons absorbed in the sensitive volume, and then reconstructing the initial direction of the photoelectrons. The primary ionization charge is multiplied and ultimately collected on a finely-pixellated ASIC specifically developed for X-ray polarimetry. The signal of individual pixels is processed independently and gain variations can be substantial, of the order of 20%. Such variations need to be equalized to correctly reconstruct the track shape, and therefore its polarization direction. The method to do such equalization is presented here and is based on the comparison between the mean charge of a pixel with respect to the other pixels for equivalent events. The method is shown to finely equalize the response of the detectors on board IXPE, allowing a better track reconstruction and energy resolution, and can in principle be applied to any imaging detector based on tracks. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.08474-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.08474) | **A machine learning-based tool for open cluster membership determination  in Gaia DR3**  |
|| M.G.J. van Groeningen, et al. -- incl., <mark>A.G.A. Brown</mark> |
|*Appeared on*| *2023-03-16*|
|*Comments*| *10 pages, 6 figures. Submitted to Astronomy & Astrophysics*|
|**Abstract**| Membership studies characterising open clusters with Gaia data, most using DR2, are so far limited at magnitude G = 18 due to astrometric uncertainties at the faint end. Our goal is to extend current open cluster membership lists with faint members and to characterise the low-mass end, which members are important for many applications, in particular for ground-based spectroscopic surveys. We use a deep neural network architecture to learn the distribution of highly reliable open cluster member stars around known clusters. After that, we use the trained network to estimate new open cluster members based on their similarities in a high-dimensional space, five-dimensional astrometry plus the three photometric bands. Due to the improved astrometric precisions of Gaia DR3 with respect to DR2, we are able to homogeneously detect new faint member stars (G > 18) for the known open cluster population. Our methodology can provide extended membership lists for open clusters down to the limiting magnitude of Gaia, which will enable further studies to characterise the open cluster population, e.g. estimation of their masses, or their dynamics. These extended membership lists are also ideal target lists for forthcoming ground-based spectroscopic surveys. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.08643-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.08643) | **Phase Curves of Kuiper Belt Objects, Centaurs, and Jupiter Family Comets  from the ATLAS Survey**  |
|| Matthew M. Dobson, et al. -- incl., <mark>Ken W. Smith</mark> |
|*Appeared on*| *2023-03-16*|
|*Comments*| *42 pages, 24 figures, accepted for publication to the Planetary Science Journal. Supplementary materials will be available in the online journal version*|
|**Abstract**| The Kuiper belt objects, the Centaurs, and the Jupiter-family comets form an evolutionary continuum of small outer Solar System objects, and their study allows us to gain insight into the history and evolution of the Solar System. Broadband photometry can be used to measure their phase curves, allowing a first-order probe into the surface properties of these objects, though limited telescope time makes measuring accurate phase curves difficult. We make use of serendipitous broadband photometry from the long-baseline, high-cadence ATLAS survey to measure the phase curves for a sample of 18 Kuiper belt objects, Centaurs, and Jupiter-family comets with unprecedentedly large datasets. We find phase curves with previously reported negative slopes become positive with increased data and are thus due to insufficient sampling of the phase curve profile, and not a real physical effect. We search for correlations between phase curve parameters, finding no strong correlations between any parameter pair, consistent with the findings of previous studies. We search for instances of cometary activity in our sample, finding a previously reported outburst by Echeclus and a new epoch of increased activity by Chiron. Applying the main belt asteroid HG1G2 phase curve model to three Jupiter-family comets in our sample with large phase angle spans, we find their slope parameters imply surfaces more consistent with those of carbonaceous main belt asteroids than silicaceous ones. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.08749-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.08749) | **Simulating image coaddition with the Nancy Grace Roman Space Telescope:  I. Simulation methodology and general results**  |
|| Christopher M. Hirata, et al. -- incl., <mark>Tianqing Zhang</mark>, <mark>Christopher W. Walter</mark> |
|*Appeared on*| *2023-03-16*|
|*Comments*| *28 pages, 19 figures, to be submitted to MNRAS*|
|**Abstract**| The upcoming Nancy Grace Roman Space Telescope will carry out a wide-area survey in the near infrared. A key science objective is the measurement of cosmic structure via weak gravitational lensing. Roman data will be undersampled, which introduces new challenges in the measurement of source galaxy shapes; a potential solution is to use linear algebra-based coaddition techniques such as Imcom that combine multiple undersampled images to produce a single oversampled output mosaic with a desired "target" point spread function (PSF). We present here an initial application of Imcom to 0.64 square degrees of simulated Roman data, based on the Roman branch of the Legacy Survey of Space and Time (LSST) Dark Energy Science Collaboration (DESC) Data Challenge 2 (DC2) simulation. We show that Imcom runs successfully on simulated data that includes features such as plate scale distortions, chip gaps, detector defects, and cosmic ray masks. We simultaneously propagate grids of injected sources and simulated noise fields as well as the full simulation. We quantify the residual deviations of the PSF from the target (the "fidelity"), as well as noise properties of the output images; we discuss how the overall tiling pattern as well as Moir\'e patterns appear in the final fidelity and noise maps. We include appendices on interpolation algorithms and the interaction of undersampling with image processing operations that may be of broader applicability. The companion paper ("Paper II") explores the implications for weak lensing analyses. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.08170-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.08170) | **Multi-scale velocity statistics in NGC 6334: deviation from a continuous  and universal turbulence cascade due to massive star formation activity**  |
|| Junhao Liu, et al. -- incl., <mark>Qizhou Zhang</mark>, <mark>Shanghuo Li</mark>, <mark>Zhi-Yun Li</mark> |
|*Appeared on*| *2023-03-16*|
|*Comments*| *14 pages, 7 figures, 1 table. Accepted by ApJ*|
|**Abstract**| We use molecular line data from ALMA, SMA, JCMT, and NANTEN2 to study the multi-scale ($\sim$15-0.005 pc) velocity statistics in the massive star formation region NGC 6334. We find that the non-thermal motions revealed by the velocity dispersion function (VDF) stay supersonic over scales of several orders of magnitudes. The multi-scale non-thermal motions revealed by different instruments do not follow the same continuous power-law, which is because the massive star formation activities near central young stellar objects have increased the non-thermal motions in small-scale and high-density regions. The magnitudes of VDFs vary in different gas materials at the same scale, where the infrared dark clump N6334S in an early evolutionary stage shows a lower level of non-thermal motions than other more evolved clumps due to its more quiescent star formation activity. We find possible signs of small-scale-driven (e.g., by gravitational accretion or outflows) supersonic turbulence in clump N6334IV with a three-point VDF analysis. Our results clearly show that the scaling relation of velocity fields in NGC 6334 deviates from a continuous and universal turbulence cascade due to massive star formation activities. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2303.08174.md
    + _build/html/tmp_2303.08174/./TOI2096_ALFOSC_vs_SDSS_conv9.png
    + _build/html/tmp_2303.08174/./radius_valley_new.png
    + _build/html/tmp_2303.08174/./kast_classify2.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\tls}{{\fontfamily{pcr}\selectfont tls} }$
$\newcommand{\ellc}{{\fontfamily{pcr}\selectfont ellc} }$
$\newcommand{\emcee}{{\fontfamily{pcr}\selectfont emcee} }$
$\newcommand{\dynesty}{{\fontfamily{pcr}\selectfont dynesty} }$
$\newcommand{\allesfitter}{{\fontfamily{pcr}\selectfont allesfitter} }$
$\newcommand{\sherlock}{{\fontfamily{pcr}\selectfont SHERLOCK} }$
$\newcommand{\matrixtk}{{\fontfamily{pcr}\selectfont MATRIX} }$
$\newcommand{\wotan}{{\fontfamily{pcr}\selectfont wotan} }$
$\newcommand{\forecaster}{{\fontfamily{pcr}\selectfont forecaster} }$
$\newcommand{\lightkurve}{{\fontfamily{pcr}\selectfont lightkurve} }$
$\newcommand{\celerite}{{\fontfamily{pcr}\selectfont celerite} }$
$\newcommand{\triceratops}{{\fontfamily{pcr}\selectfont TRICERATOPS} }$
$\newcommand{\orcidauthorA}{0000-0003-1572-7707}$
$\newcommand{\orcidauthorB}{0000-0002-3164-9086}$
$\newcommand{\orcidauthorC}{0000-0003-1462-7739}$
$\newcommand{\orcidauthorD}{0000-0002-8219-9505}$
$\newcommand{\orcidauthorE}{0000-0002-4715-9460}$
$\newcommand{\orcidauthorF}{0000-0002-5370-7494}$
$\newcommand{\orcidauthorG}{0000-0001-8504-5862}$
$\newcommand{\orcidauthorH}{0000-0002-7008-6888}$
$\newcommand{\orcidauthorI}{0000-0002-6523-9536}$
$\newcommand{\orcidauthorJ}{0000-0002-3627-1676}$
$\newcommand{\orcidauthorK}{0000-0002-3481-9052}$
$\newcommand{\orcidauthorL}{0000-0003-2144-4316}$
$\newcommand{\orcidauthorM}{0000-0003-2094-9128}$
$\newcommand{\orcidauthorN}{0000-0002-2532-2853}$
$\newcommand{\orcidauthorO}{0000-0002-8052-3893}$
$\newcommand{\orcidauthorP}{0000-0002-2214-9258}$
$\newcommand{\orcidauthorQ}{0000-0003-1464-9276}$
$\newcommand{\orcidauthorR}{0000-0001-9892-2406}$
$\newcommand{\orcidauthorS}{0000-0002-9355-5165}$
$\newcommand{\orcidauthorT}{0000-0002-3937-630X}$
$\newcommand{\orcidauthorU}{0000-0002-9113-7162}$
$\newcommand{\orcidauthorV}{0000-0001-9800-6248}$
$\newcommand{\orcidauthorW}{0000-0002-4296-2246}$
$\newcommand{\orcidauthorY}{0000-0002-7486-6726}$
$\newcommand{\orcidauthorX}{0000-0002-3938-4211}$
$\newcommand{\orcidauthorZ}{0000-0002-3164-9086}$
$\newcommand{\orcidauthora}{0000-0001-8923-488X}$
$\newcommand{\orcidauthorb}{0000-0001-6108-4808}$
$\newcommand{\orcidauthorc}{0000-0001-9911-7388}$
$\newcommand{\orcidauthord}{0000-0002-3012-0316}$
$\newcommand{\orcidauthore}{0000-0003-0242-0044}$
$\newcommand{\orcidauthorf}{0000-0002-9807-5435}$
$\newcommand{\orcidauthorg}{0000-0002-5510-8751}$
$\newcommand{\orcidauthorh}{0000-0002-8039-194X}$
$\newcommand{\orcidauthori}{0000-0003-0030-332X}$
$\newcommand{\orcidauthorj}{0000-0002-7084-0529}$
$\newcommand{\orcidauthork}{0000-0002-8388-6040}$
$\newcommand{\orcidauthorl}{0000-0001-5559-7850}$
$\newcommand{\orcidauthorm}{0000-0003-3649-8384}$
$\newcommand{\orcidauthorn}{0000-0003-3101-917X}$
$\newcommand{\orcidauthoro}{0000-0002-3321-4924}$
$\newcommand{\orcidauthorp}{0000-0001-9291-5555}$
$\newcommand{\orcidauthorr}{0000-0003-3714-5855}$
$\newcommand{\orcidauthors}{0000-0002-9350-830X}$
$\newcommand{\orcidauthort}{0000-0003-2007-6144}$
$\newcommand{\orcidauthoru}{0000-0002-4671-2957}$
$\newcommand{\orcidauthorv}{0000-0002-5494-3237}$
$\newcommand{\orcidauthorw}{0000-0002-7240-8473}$
$\newcommand{\orcidauthorx}{0000-0003-3767-7085}$
$\newcommand{\arraystretch}{1.15}$
$\newcommand{\arraystretch}{1.15}$
$\newcommand{\arraystretch}{1.0}$
$\newcommand{\arraystretch}{1.15}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\tls}{{\fontfamily{pcr}\selectfont tls} }$
$\newcommand{\ellc}{{\fontfamily{pcr}\selectfont ellc} }$
$\newcommand{\emcee}{{\fontfamily{pcr}\selectfont emcee} }$
$\newcommand{\dynesty}{{\fontfamily{pcr}\selectfont dynesty} }$
$\newcommand{\allesfitter}{{\fontfamily{pcr}\selectfont allesfitter} }$
$\newcommand{\sherlock}{{\fontfamily{pcr}\selectfont SHERLOCK} }$
$\newcommand{\matrixtk}{{\fontfamily{pcr}\selectfont MATRIX} }$
$\newcommand{\wotan}{{\fontfamily{pcr}\selectfont wotan} }$
$\newcommand{\forecaster}{{\fontfamily{pcr}\selectfont forecaster} }$
$\newcommand{\lightkurve}{{\fontfamily{pcr}\selectfont lightkurve} }$
$\newcommand{\celerite}{{\fontfamily{pcr}\selectfont celerite} }$
$\newcommand{\triceratops}{{\fontfamily{pcr}\selectfont TRICERATOPS} }$
$\newcommand{\orcidauthorA}{0000-0003-1572-7707}$
$\newcommand{\orcidauthorB}{0000-0002-3164-9086}$
$\newcommand{\orcidauthorC}{0000-0003-1462-7739}$
$\newcommand{\orcidauthorD}{0000-0002-8219-9505}$
$\newcommand{\orcidauthorE}{0000-0002-4715-9460}$
$\newcommand{\orcidauthorF}{0000-0002-5370-7494}$
$\newcommand{\orcidauthorG}{0000-0001-8504-5862}$
$\newcommand{\orcidauthorH}{0000-0002-7008-6888}$
$\newcommand{\orcidauthorI}{0000-0002-6523-9536}$
$\newcommand{\orcidauthorJ}{0000-0002-3627-1676}$
$\newcommand{\orcidauthorK}{0000-0002-3481-9052}$
$\newcommand{\orcidauthorL}{0000-0003-2144-4316}$
$\newcommand{\orcidauthorM}{0000-0003-2094-9128}$
$\newcommand{\orcidauthorN}{0000-0002-2532-2853}$
$\newcommand{\orcidauthorO}{0000-0002-8052-3893}$
$\newcommand{\orcidauthorP}{0000-0002-2214-9258}$
$\newcommand{\orcidauthorQ}{0000-0003-1464-9276}$
$\newcommand{\orcidauthorR}{0000-0001-9892-2406}$
$\newcommand{\orcidauthorS}{0000-0002-9355-5165}$
$\newcommand{\orcidauthorT}{0000-0002-3937-630X}$
$\newcommand{\orcidauthorU}{0000-0002-9113-7162}$
$\newcommand{\orcidauthorV}{0000-0001-9800-6248}$
$\newcommand{\orcidauthorW}{0000-0002-4296-2246}$
$\newcommand{\orcidauthorY}{0000-0002-7486-6726}$
$\newcommand{\orcidauthorX}{0000-0002-3938-4211}$
$\newcommand{\orcidauthorZ}{0000-0002-3164-9086}$
$\newcommand{\orcidauthora}{0000-0001-8923-488X}$
$\newcommand{\orcidauthorb}{0000-0001-6108-4808}$
$\newcommand{\orcidauthorc}{0000-0001-9911-7388}$
$\newcommand{\orcidauthord}{0000-0002-3012-0316}$
$\newcommand{\orcidauthore}{0000-0003-0242-0044}$
$\newcommand{\orcidauthorf}{0000-0002-9807-5435}$
$\newcommand{\orcidauthorg}{0000-0002-5510-8751}$
$\newcommand{\orcidauthorh}{0000-0002-8039-194X}$
$\newcommand{\orcidauthori}{0000-0003-0030-332X}$
$\newcommand{\orcidauthorj}{0000-0002-7084-0529}$
$\newcommand{\orcidauthork}{0000-0002-8388-6040}$
$\newcommand{\orcidauthorl}{0000-0001-5559-7850}$
$\newcommand{\orcidauthorm}{0000-0003-3649-8384}$
$\newcommand{\orcidauthorn}{0000-0003-3101-917X}$
$\newcommand{\orcidauthoro}{0000-0002-3321-4924}$
$\newcommand{\orcidauthorp}{0000-0001-9291-5555}$
$\newcommand{\orcidauthorr}{0000-0003-3714-5855}$
$\newcommand{\orcidauthors}{0000-0002-9350-830X}$
$\newcommand{\orcidauthort}{0000-0003-2007-6144}$
$\newcommand{\orcidauthoru}{0000-0002-4671-2957}$
$\newcommand{\orcidauthorv}{0000-0002-5494-3237}$
$\newcommand{\orcidauthorw}{0000-0002-7240-8473}$
$\newcommand{\orcidauthorx}{0000-0003-3767-7085}$
$\newcommand{\arraystretch}{1.15}$
$\newcommand{\arraystretch}{1.15}$
$\newcommand{\arraystretch}{1.0}$
$\newcommand{\arraystretch}{1.15}$</div>



<div id="title">

# A super-Earth and a mini-Neptune near the 2:1 MMR straddling the radius valley around the nearby mid-M dwarf TOI-2096

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.08174-b31b1b.svg)](https://arxiv.org/abs/2303.08174)<mark>Appeared on: 2023-03-16</mark> - _25 pages, 21 figures. Aceptted for publication in Astronomy & Astrophysics_

</div>
<div id="authors">

F. J. Pozuelos, et al. -- incl., <mark><mark>R. D. Wells</mark></mark>, <mark><mark>G. Torres</mark></mark>

</div>
<div id="abstract">

**Abstract:** Several planetary formation models have been proposed to explain the observed abundance and variety of compositions of super-Earths and mini-Neptunes. In this context, multitransiting systems orbiting low-mass stars whose planets are close to the radius valley are benchmark systems, which help to elucidate which formation model dominates. We report the discovery, validation, and initial characterization of one such system, TOI-2096 (TIC 142748283), a two-planet system composed of a super-Earth and a mini-Neptune hosted by a mid-type M dwarf located 48 pc away. We characterized the host star by combining optical spectra, analyzing its broadband spectral energy distribution, and using evolutionary models for low-mass stars. Then, we derived the planetary properties by modeling the photometric data from TESS and ground-based facilities. In addition, we used archival data, high-resolution imaging, and statistical validation to support our planetary interpretation. We found that the stellar properties of TOI-2096 correspond to a dwarf star of spectral type M4 $\pm$ 0.5. It harbors a super-Earth (R=1.24 $\pm{0.07}$ R $_{\oplus}$ ) and a mini-Neptune (R=1.90 $\pm{0.09}$ R $_{\oplus}$ ) in likely slightly eccentric orbits with orbital periods of 3.12 d and 6.39 d, respectively. These orbital periods are close to the first-order 2:1 mean-motion resonance (MMR), a configuration that may lead to measurable transit timing variations (TTVs). We computed the expected TTVs amplitude for each planet and found that they might be measurable with high-precision photometry delivering mid-transit times with accuracies of $\lesssim$ 2 min. Moreover, we conclude that measuring the planetary masses via radial velocities (RVs) could also be possible. Lastly, we found that these planets are among the best in their class to conduct atmospheric studies using the NIRSpec/Prism onboard the James Webb Space Telescope (JWST). The properties of this system make it a suitable candidate for further studies, particularly for mass determination using RVs and/or TTVs, decreasing the scarcity of systems that can be used to test planetary formation models around low-mass stars.

</div>

<div id="div_fig1">

<img src="tmp_2303.08174/./TOI2096_ALFOSC_vs_SDSS_conv9.png" alt="Fig1" width="100%"/>

**Figure 1. -** NOT/ALFOSC optical spectrum of TOI-2096 (black) compared to SDSS M3, M4, and M5 templates \citet{Bochanski2007} and \citet{kesseli:2017:16}(colored). The SDSS spectra were convolved using a Gaussian function to match the ALFOSC spectrum resolution. The spectra were normalized at $\sim$7500 Å  and shifted by a constant. Some remarkable absorption bands and lines are labeled in gray. (*fig:ALFOSCspT*)

</div>
<div id="div_fig2">

<img src="tmp_2303.08174/./radius_valley_new.png" alt="Fig11" width="100%"/>

**Figure 11. -** The left panel depicts the planet radius--orbital period plane, containing all the transiting planets with sizes <4 R$_{\oplus}$ orbiting low-mass stars (stellar mass < 0.70 M$_{\odot}$). The green contours represent the density distribution of all of these planets. The gray dots are planets with density estimations better than 30$\%$. The dashed line refers to the prediction using thermally driven mass-loss mechanisms, such as photoevaporation and core-powered mass-loss. On the other hand, the solid line represents the prediction from the gas-poor formation model.
Planets inside the region limited by these two lines are referred to as `keystone' planets. The red and the blue dots refer to the planets TOI-2096 b and c, respectively. The right panel displays the radius frequency, showing the radius valley and the location of planets b (red band) and c (blue band) encompassing it. (*fig:radiusvalley*)

</div>
<div id="div_fig3">

<img src="tmp_2303.08174/./kast_classify2.png" alt="Fig2" width="100%"/>

**Figure 2. -** 
        Shane/Kast red optical spectrum of TOI-2096 (black), compared to an M4 template \citep[magenta]{kesseli:2017:16}. Spectra are normalized in the 7400--7500 {Å} region, and major absorption features are labeled, including resolved lines.
        The inset box shows a close-up of the region hosting H$\alpha$ and Li i features. The former is obscured by an unfortunate cosmic ray strike (unmasked spectral data in grey); the latter is not detected.
     (*fig:kast*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

197  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
